In [1]:
%%capture
!pip install ultralytics

In [2]:
from ultralytics import YOLO

In [3]:
%%capture
!wget https://www.dropbox.com/scl/fi/maqbuw2e0s7ywa63dofun/bestSmall1920.pt?rlkey=n0pqsnjgycak6v7jx3qzhladt&st=04t3nonx&dl=0
!mv bestSmall1920.pt?rlkey=n0pqsnjgycak6v7jx3qzhladt bestSmall1920.pt

In [4]:
%%capture
!wget https://www.dropbox.com/scl/fi/3mwyjefpr86kqktwrs1dh/HeadBest.pt?rlkey=b1h3rsr1z4cb82yh2najhvmrj&st=k4l2wefe&dl=0
!mv HeadBest.pt?rlkey=b1h3rsr1z4cb82yh2najhvmrj HeadBest.pt

In [5]:
# Load a pretrained YOLO model (recommended for training)
model = YOLO('bestSmall1920.pt')

In [6]:
# Load a pretrained YOLO model (recommended for training)
modelHeads = YOLO('HeadBest.pt')

In [7]:
%%capture
# Descargar el archivo ZIP desde la URL
!wget https://www.dropbox.com/scl/fi/wgyl8ibb6rgo04ezuhzd5/Suppl_1.zip?rlkey=rmme7la6j6593wuacwiap8j46&st=qvcm3x1k&dl=0

# Crear la carpeta "Videos" si no existe
!mkdir -p Videos

!mv Suppl_1.zip?rlkey=rmme7la6j6593wuacwiap8j46 Suppl_1.zip

# Descomprimir el archivo ZIP en la carpeta "Videos"
!unzip -o Suppl_1.zip -d Videos

In [8]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from collections import defaultdict
import os

cv2.imshow = cv2_imshow


In [9]:
import os
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO  # Asegúrate de tener instalado YOLOv8
from scipy.spatial.distance import euclidean

# Directorio de entrada y salida
input_folder = "Videos/"
output_folder = "output_videos/"

# Crear el directorio de salida si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Listar los archivos de video en el directorio de entrada
video_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

# Iterar sobre cada archivo de video
for video_file in video_files:
    # Abrir el archivo de video de entrada
    video_path = os.path.join(input_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    # Verificar si la apertura del video fue exitosa
    if not cap.isOpened():
        print(f"Error al abrir el archivo de video: {video_file}")
        continue

    # Obtener información del video de entrada
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (frame_width, frame_height)

    # Crear el objeto VideoWriter para el video de salida
    output_video_file = os.path.join(output_folder, f"{os.path.splitext(video_file)[0]}_Out.avi")
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Formato de compresión de vídeo
    out = cv2.VideoWriter(output_video_file, fourcc, fps, frame_size)

    # Store the track history
    track_history = defaultdict(lambda: [])
    head_track_history = defaultdict(lambda: [])
    last_head_position = defaultdict(lambda: None)

    model = YOLO('bestSmall1920.pt')

    # Loop through the video frames
    while cap.isOpened():
        # Leer un cuadro del video
        success, frame = cap.read()

        if success:
            # Ejecutar el seguimiento YOLOv8 en el cuadro, persistiendo los tracks entre cuadros
            results = model.track(frame, persist=True)

            # Verificar si se detectaron resultados
            if results and len(results) > 0 and len(results[0].boxes) > 0:
                # Obtener las cajas y los IDs de seguimiento
                boxes = results[0].boxes.xywh.cpu().numpy()
                track_ids = results[0].boxes.id.cpu().numpy()

                # Visualizar los resultados en el cuadro
                annotated_frame = results[0].plot()

                # Dibujar las líneas de seguimiento
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))  # x, y centro
                    if len(track) > 80:  # mantiene 80 tracks por 80 cuadros
                        track.pop(0)

                    # ************* Apartado a modificar *************
                    ################################################
                    # Recortar la región de interés (ROI) del cuadro original
                    roi = frame[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)]

                    # Inicializar variable para almacenar la nueva posición de la cabeza
                    new_head_position = None

                    # Verificar que el ROI no esté vacío
                    if roi.size != 0:
                        # Ejecutar el modelo de detección de cabezas en el ROI
                        head_results = modelHeads.predict(roi)

                        # Obtener las detecciones de cabezas
                        if head_results and len(head_results[0].boxes) > 0:
                            head_boxes = head_results[0].boxes.xywh.cpu().numpy()
                            head_classes = head_results[0].boxes.cls.cpu().numpy()
                            for head_box, head_cls in zip(head_boxes, head_classes):
                                if head_cls == 0:  # Si la clase es 0
                                    hx, hy, hw, hh = head_box
                                    head_center_x = x - w/2 + hx
                                    head_center_y = y - h/2 + hy

                                    new_head_position = (float(head_center_x), float(head_center_y))

                                    # Dibujar la caja de la cabeza en el cuadro original
                                    cv2.rectangle(frame,
                                                  (int(head_center_x - hw/2), int(head_center_y - hh/2)),
                                                  (int(head_center_x + hw/2), int(head_center_y + hh/2)),
                                                  (255, 0, 0), 2)

                    # Si no se detectó una nueva cabeza, utilizar la última posición conocida
                    if new_head_position is None and last_head_position[track_id] is not None:
                        new_head_position = last_head_position[track_id]
                        dist = euclidean(new_head_position, last_head_position[track_id])
                        if  dist > 30:
                            new_head_position = last_head_position[track_id]
                            print("Distancia=", dist)  # Si la distancia es mayor a 30, no se actualiza la posición

                    # Actualizar la última posición conocida de la cabeza
                    if new_head_position is not None:
                        head_track = head_track_history[track_id]
                        head_track.append(new_head_position)
                        last_head_position[track_id] = new_head_position  # Actualizar la última posición conocida
                        if len(head_track) > 80:  # mantener 80 tracks por 80 cuadros
                            head_track.pop(0)
                    ################################################

                    # Dibujar las líneas de seguimiento
                    points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [points], isClosed=False, color=(0, 255, 255), thickness=3)

                    # Dibujar las líneas de seguimiento de las cabezas
                    head_points = np.array(head_track_history[track_id], dtype=np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [head_points], isClosed=False, color=(255, 0, 0), thickness=3)

                # Escribir el cuadro procesado en el archivo de salida
                out.write(frame)
            else:
                # Si no se detectaron resultados, escribir el cuadro original en el archivo de salida
                out.write(frame)

        else:
            # Terminar la ejecución si se alcanza el final del video
            break

    # Liberar los recursos
    cap.release()
    out.release()


Se truncaron las últimas líneas 5000 del resultado de transmisión.
0: 640x640 (no detections), 2042.8ms
Speed: 4.5ms preprocess, 2042.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 2489.1ms
Speed: 8.0ms preprocess, 2489.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Head, 1581.7ms
Speed: 4.1ms preprocess, 1581.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Head, 1534.2ms
Speed: 3.7ms preprocess, 1534.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Tail, 1596.9ms
Speed: 3.8ms preprocess, 1596.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x608 (no detections), 1524.4ms
Speed: 3.8ms preprocess, 1524.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 608)

0: 640x640 (no detections), 1557.2ms
Speed: 5.1ms preprocess, 1557.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0